# Explanation of `tf.py_func` with example

The `tf.py_func` operator enables us to run arbitrary Python code in the middle of a TensorFlow graph. This might be needed to wrap custom defined NumPy operators for which no equivalent TensorFlow operator exists. One situation which comes to my mind, where one might have to use this operator is for finding the gradient of the matrix determinant. Gradient for this operation isn't defined in TensorFlow. 

The `tf.py_func` operator, however, should be used only when no other alternative exists. Firstly, because it essentially injects a python function into the graph thereby making the code slower than the C code most low-level functions are written in. Secondly, because the `tf.py_func()` operation will only run on CPU. If one is using TensorFlow in a distributed setting, the `tf.py_func()` operation must be placed on a CPU device in the same process as the client.

The `tf.py_func(func, inp, Tout)` operator creates a TensorFlow operation that calls a Python function, `func` on a list of tensors `inp`. 

The following snippet is taken from __[TF documentation](https://www.tensorflow.org/api_docs/python/tf/py_func)__ of this operator.

```python
def my_func(x):
  # x will be a numpy array with the contents of the placeholder below
  return np.sinh(x)
inp = tf.placeholder(tf.float32)
y = tf.py_func(my_func, [inp], tf.float32)
```

### Simple ReLU nonlinear activation in TensorFlow as a Python op

Recall that for deep neural networks, a common activation function is $ReLU(x) = max(0, x)$. If you graph $y = ReLU(x)$ you can see that the function is mostly differentiable. If $x$ is greater than 0 the derivative is 1 and if $x$ is less than zero the derivative is 0. However, at $x = 0$, the derivative does not exist.

I have borrowed the following code snippet from __[here](https://github.com/vahidk/EffectiveTensorflow#python_ops)__. In the first part, we have the implementation of ReLU. In the second part we will cross check the gradient calculation using TF's gradient checker. 

#### ReLU as a Python op

In [3]:
import numpy as np
import tensorflow as tf
import uuid

def relu(inputs):
    # Define the op in python
    def _relu(x):
        return np.maximum(x, 0.)

    # Define the op's gradient in python
    def _relu_grad(x):
        return np.float32(x > 0)

    # An adapter that defines a gradient op compatible with TensorFlow
    def _relu_grad_op(op, grad):
        x = op.inputs[0]
        x_grad = grad * tf.py_func(_relu_grad, [x], tf.float32)
        return x_grad

    # Register the gradient with a unique id
    grad_name = "MyReluGrad_" + str(uuid.uuid4())
    tf.RegisterGradient(grad_name)(_relu_grad_op)

    # Override the gradient of the custom op
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": grad_name}):
        output = tf.py_func(_relu, [inputs], tf.float32)
    return output

#### TensorFlow's gradient checker to cross check gradient

In [2]:
x = tf.random_normal([10])
y = relu(x * x)

with tf.Session():
    diff = tf.test.compute_gradient_error(x, [10], y, [10])
    print(diff)

4.57763671875e-05
